In [ ]:
# ✅ 설치된 requirements.txt로 환경 구성
!pip install -r https://raw.githubusercontent.com/sun2141/stable-diffusion-colab/main/requirements.txt

In [ ]:
# ✅ 모델 임포트 시작
from diffusers import StableDiffusionPipeline, StableDiffusionImg2ImgPipeline
from PIL import Image
import torch
import gradio as gr
import numpy as np

In [ ]:
# ✅ 모델 불러오기
pipe_txt2img = StableDiffusionPipeline.from_pretrained(
    "CompVis/stable-diffusion-v1-4", torch_dtype=torch.float16
).to("cuda")

In [ ]:
pipe_img2img = StableDiffusionImg2ImgPipeline.from_pretrained(
    "CompVis/stable-diffusion-v1-4", torch_dtype=torch.float16
).to("cuda")

In [ ]:
# ✅ 생성 함수
def infer(prompt, init_image, strength):
    if init_image is not None:
        if isinstance(init_image, np.ndarray):
            init_image = Image.fromarray(init_image)
        elif not isinstance(init_image, Image.Image):
            raise ValueError(f"Unsupported image type: {type(init_image)}")

In [ ]:
init_image = init_image.resize((512, 512))

In [ ]:
with torch.autocast("cuda"):
            result = pipe_img2img(
                prompt=[prompt],
                image=init_image,
                strength=strength,
                guidance_scale=7.5
            )
            return result.images
    else:
        with torch.autocast("cuda"):
            result = pipe_txt2img(
                prompt=[prompt],
                guidance_scale=7.5
            )
            return result.images

In [ ]:
# ✅ Gradio 인터페이스 구성
with gr.Blocks() as demo:
    gr.Markdown("# 🎨 Stable Diffusion Image Generator")

In [ ]:
prompt_input = gr.Textbox(label="프롬프트", value="a cushion on a car seat")
    init_image = gr.Image(label="초기 이미지 (선택)", type="numpy")
    strength = gr.Slider(label="유지 강도", minimum=0.0, maximum=1.0, value=0.75)
    gallery = gr.Gallery(label="결과 이미지")

In [ ]:
run_button = gr.Button("이미지 생성")

In [ ]:
run_button.click(fn=infer, inputs=[prompt_input, init_image, strength], outputs=[gallery])

In [ ]:
demo.launch(debug=True)